In [2]:
import re
import pandas as pd
from tqdm import tqdm

**1. Finding the Unique Words**

In [3]:
with open('big.txt','r') as fd:
    lines = fd.readlines()
    words = []
    for line in lines:
        words += re.findall('\w+',line.lower())
        
print(len(words))
vocab = list(set(words))
print(len(vocab))

1115585
32198


**2. Finding the Probability Distribution**

In [4]:
word_probability = {}

for word in tqdm(vocab):
    word_probability[word] = float(words.count(word)/len(words))

  0%|          | 0/32198 [00:00<?, ?it/s]

100%|██████████| 32198/32198 [11:37<00:00, 46.15it/s]


**3. Text Preprocessing**

Splitting

In [5]:
def split(word):  
    parts = []
    for i in range(len(word) + 1):
        parts += [(word[ : i], word[i : ])]
    return parts

**3.1) Delete**<br>

'loave' -> 'love'

In [6]:
def delete(word):
    
    output = []
    for l,r in split(word):
        output.append(l + r[1:])
    return output

delete('loave')

['oave', 'lave', 'love', 'loae', 'loav', 'loave']

**3.2) Swap**<br>

'lvoe' -> 'love'

In [7]:
def swap(word):
        
    output = []    
    for l,r in split(word):
        if (len(r) > 1):
            output.append(l + r[1] + r[0] + r[2:])
    return output
            
swap('lvoe')

['vloe', 'love', 'lveo']

**3.3) Replace**<br>


'lave' -> 'love'

In [8]:
def replace(word):
    
    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []    

    for l,r in split(word):
        for char in characters:
            output.append(l + char +  r[1:])
    return output

len(replace('lave'))

130

**3.4) Insert**<br>

'lve' -> 'love'

In [9]:
def insert(word):

    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []

    for l,r in split(word):
        for char in characters:
            output.append(l + char + r)

    return output

len(insert('lve'))

104

**4. Finding the Prediction (Level - 1)**

**4.1) Combining Possible Words**

In [10]:
def edit(word):   
    return list(set(insert(word) + delete(word) + swap(word) + replace(word)))

**4.2) Predicting the Word**

In [11]:
def spell_check_edit_1(word, count = 5):
    
    output = []
    suggested_words = edit(word)
    
    for wrd in suggested_words:        
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
            
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)

In [12]:
spell_check_edit_1('famili')

['family']

**5. Finding the Prediction (Level - 2)**<br>

**5.1) Combining Possible Words**

In [13]:
def spell_check_edit_2(word, count = 5):
    
    output = []
    suggested_words = edit(word)       # Level one Edit
    
    for e1 in edit(word):
        suggested_words += edit(e1)    # Second Level Edit 
    
    suggested_words = list(set(suggested_words))
    
    for wrd in suggested_words:
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)
        


In [14]:
spell_check_edit_2('fameli')

['family', 'namely', 'fame', 'camelia', 'camel']